## 准备数据

In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [11]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [12]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        # 28*28的图片，200个神经元，10种可能的结果
        self.W1=tf.Variable(shape=[28*28,200], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 200],minval=-0.1, maxval=0.1))
        self.b1=tf.Variable(shape=[200],dtype=tf.float32, initial_value=tf.zeros(200))
        self.W2=tf.Variable(shape=[200,10], dtype=tf.float32,initial_value=tf.random.uniform(shape=[200, 10],minval=-0.1, maxval=0.1))
        self.b2=tf.Variable(shape=[10],dtype=tf.float32, initial_value=tf.zeros(10))
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_=tf.reshape(x,shape=[-1,28*28])
        y=tf.tanh(tf.matmul(x_,self.W1)+self.b1)  # 用tanh激活函数
        logits=tf.matmul(y,self.W2)+self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [13]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [14]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3323243 ; accuracy 0.09126667
epoch 1 : loss 2.3210044 ; accuracy 0.096033335
epoch 2 : loss 2.3098755 ; accuracy 0.10096667
epoch 3 : loss 2.2989278 ; accuracy 0.1065
epoch 4 : loss 2.2881532 ; accuracy 0.11265
epoch 5 : loss 2.2775426 ; accuracy 0.11928333
epoch 6 : loss 2.2670891 ; accuracy 0.12686667
epoch 7 : loss 2.2567844 ; accuracy 0.1346
epoch 8 : loss 2.246622 ; accuracy 0.14373334
epoch 9 : loss 2.2365954 ; accuracy 0.15281667
epoch 10 : loss 2.2266989 ; accuracy 0.16261667
epoch 11 : loss 2.2169266 ; accuracy 0.17275
epoch 12 : loss 2.207273 ; accuracy 0.18343334
epoch 13 : loss 2.1977332 ; accuracy 0.19516666
epoch 14 : loss 2.1883025 ; accuracy 0.2063
epoch 15 : loss 2.1789768 ; accuracy 0.2181
epoch 16 : loss 2.1697514 ; accuracy 0.22946666
epoch 17 : loss 2.1606228 ; accuracy 0.23985
epoch 18 : loss 2.151587 ; accuracy 0.25096667
epoch 19 : loss 2.142641 ; accuracy 0.26145
epoch 20 : loss 2.1337812 ; accuracy 0.27226666
epoch 21 : loss 2.1250048 ; accur